import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

PATH = "data/ds_salaries.csv"

     create local SparkSession

In [42]:
ss = SparkSession.builder.getOrCreate()
ss.sparkContext.setLogLevel(logLevel="ERROR")

read csv with inferschema

In [3]:
df_inferschema = ss.read.csv(path=PATH, header=True, inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
df_inferschema = ss.read.csv(path=PATH, header=True, inferSchema=True)

write schema of scv on screen

In [5]:
df_inferschema.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [43]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('work_year', StringType(), True),
    StructField('experience_level', StringType(), True),
    StructField('employment_type', StringType(), True),
    StructField('job_title', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('salary_currency', StringType(), True),
    StructField('salary_in_usd', IntegerType(), True),
    StructField('employee_residence', StringType(), True),
    StructField('remote_ratio', IntegerType(), True),
    StructField('company_location', StringType(), True),
    StructField('company_size', StringType(), True),
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [44]:
df = ss.read.csv(path=PATH, header=True, schema=schema)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [45]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [46]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [47]:
display(df.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [48]:
df_job_title = df.select('job_title').distinct().toPandas()

print all rows from df_job_titles without truncating jobs

In [49]:
df_job_title

,job_title
0,3D Computer Vision Researcher
1,Lead Data Engineer
2,Head of Machine Learning
3,Data Specialist
4,Data Analytics Lead
5,Machine Learning Scientist
6,Lead Data Analyst
7,Data Engineering Manager
8,Staff Data Scientist
9,ETL Developer


create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [50]:
df_analytic = df.filter(col('salary_currency') == 'USD').groupby('job_title').agg(
    avg('salary_in_usd').alias('avg_salary'),
    min('salary_in_usd').alias('min_salary'),
    max('salary_in_usd').alias('max_salary')
    )

print all rows from df_analytic without trancating jobs

In [51]:
df_analytic.toPandas()

,job_title,avg_salary,min_salary,max_salary
0,Lead Data Engineer,154250.000000,56000,276000
1,Data Specialist,165000.000000,165000,165000
2,Data Analytics Lead,405000.000000,405000,405000
3,Machine Learning Scientist,158412.500000,12000,260000
4,Lead Data Analyst,128500.000000,87000,170000
5,Data Engineering Manager,159000.000000,150000,174000
6,Staff Data Scientist,105000.000000,105000,105000
7,Director of Data Engineering,200000.000000,200000,200000
8,Product Data Analyst,20000.000000,20000,20000
9,Principal Data Scientist,255500.000000,151000,416000


now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [52]:
window = Window.partitionBy().orderBy(col('avg_salary').desc())
df_analytic_sorted = df_analytic.withColumn('row_id', row_number().over(window))

print all data from df_analytic

In [53]:
df_analytic_sorted.toPandas()

,job_title,avg_salary,min_salary,max_salary,row_id
0,Data Analytics Lead,405000.000000,405000,405000,1
1,Principal Data Engineer,328333.333333,185000,600000,2
2,Financial Data Analyst,275000.000000,100000,450000,3
3,ML Engineer,263000.000000,256000,270000,4
4,Principal Data Scientist,255500.000000,151000,416000,5
5,Director of Data Science,247666.666667,168000,325000,6
6,Applied Data Scientist,238000.000000,157000,380000,7
7,Head of Data,221666.666667,200000,235000,8
8,Director of Data Engineering,200000.000000,200000,200000,9
9,Machine Learning Infrastructure Engineer,195000.000000,195000,195000,10


    it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [69]:
from pyspark.sql.functions import stddev

cols = df_analytic_sorted.columns
df_analytic = df_analytic_sorted.select([cols[-1]] + cols[:-1])

print df_analytic now

In [70]:
df_analytic.toPandas()

,row_id,job_title,avg_salary,min_salary,max_salary
0,1,Data Analytics Lead,405000.000000,405000,405000
1,2,Principal Data Engineer,328333.333333,185000,600000
2,3,Financial Data Analyst,275000.000000,100000,450000
3,4,ML Engineer,263000.000000,256000,270000
4,5,Principal Data Scientist,255500.000000,151000,416000
5,6,Director of Data Science,247666.666667,168000,325000
6,7,Applied Data Scientist,238000.000000,157000,380000
7,8,Head of Data,221666.666667,200000,235000
8,9,Director of Data Engineering,200000.000000,200000,200000
9,10,Machine Learning Infrastructure Engineer,195000.000000,195000,195000


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [76]:
window = Window.partitionBy('experience_level').orderBy(col('salary_in_usd').desc())
df_exp_lvl = df.withColumn('salary_order', row_number().over(window))\
    .filter(col('salary_order') == 1)\
    .drop('salary_order')\
    .withColumnRenamed('salary', 'biggest_salary')

print here df_exp_lvl

In [77]:
df_exp_lvl.toPandas()

,id,work_year,experience_level,employment_type,job_title,biggest_salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,37,2020,EN,FT,Machine Learning Engineer,250000,USD,250000,US,50,US,L
1,252,2021,EX,FT,Principal Data Engineer,600000,USD,600000,US,100,US,L
2,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M
3,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [111]:
window = Window.partitionBy('experience_level').orderBy(col('salary').desc())

df_best = df.withColumn('salary_order', row_number().over(window)).\
    filter(col('salary_order') == 1).\
    drop('salary_order')\
    .select('id', 'experience_level', 'salary', 'employee_residence')

print df_best

In [112]:
df_best.toPandas()

,id,experience_level,salary,employee_residence
0,16,EN,4450000,JP
1,384,EX,6000000,IN
2,177,MI,30400000,CL
3,285,SE,7000000,IN


drop duplicates if exist by experience_level

In [113]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [114]:
df_best.toPandas()

,id,experience_level,salary,employee_residence
0,16,EN,4450000,JP
1,384,EX,6000000,IN
2,177,MI,30400000,CL
3,285,SE,7000000,IN


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [130]:
df_new_best = df_best.drop('id')\
    .withColumn('grade', when(df.experience_level == 'MI', 'middle')
                        .when(df.experience_level == 'SE', 'senior'))

print df_new_best

In [131]:
df_new_best.toPandas()

,experience_level,salary,employee_residence,grade
0,EN,4450000,JP,None
1,EX,6000000,IN,None
2,MI,30400000,CL,middle
3,SE,7000000,IN,senior


write df_new_best like 1.csv and load then it to df_final

In [134]:
df_new_best.write.csv('1.csv', header=True)

С нормальным именем не сохранить, внутри используется hadoop file format. [Источник.](https://stackoverflow.com/questions/41990086/specifying-the-filename-when-saving-a-dataframe-as-a-csv)
Но как тогда read.csv() находит нужный csv? По папке?

In [149]:
df_final = ss.read.csv('1.csv', header=True, inferSchema=True)

print df_final

In [150]:
df_final.toPandas()

,experience_level,salary,employee_residence,grade
0,EN,4450000,JP,None
1,EX,6000000,IN,None
2,MI,30400000,CL,middle
3,SE,7000000,IN,senior


filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [151]:
df_final = df_final.dropna()\
    .join(df, ['salary', 'experience_level']) # лучше, чем просто join(condition), потому что нет дубликатов по столбцам

print df_final

In [152]:
df_final.toPandas()

,salary,experience_level,employee_residence,grade,id,work_year,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,30400000,MI,CL,middle,177,2021,FT,Data Scientist,CLP,40038,CL,100,CL,L
1,7000000,SE,IN,senior,285,2021,FT,Data Science Manager,INR,94665,IN,50,IN,L


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [155]:
temp_var = df_final
temp_var.select(df_final.salary_in_usd).show()

+-------------+
|salary_in_usd|
+-------------+
|        40038|
|        94665|
+-------------+



It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system